## DSpy Prompt Optimisation for RAG Systems

Retrieval-augmented generation (RAG) is an approach that allows LLMs to tap into a large corpus of knowledge from sources and query its knowledge store to find relevant passages/content and produce a well-refined response.

RAG ensures LLMs can dynamically utilize real-time knowledge even if not originally trained on the subject and give thoughtful answers. DSpy provides a compiler architecture to optimise a prompt (aka teleprompers) which offer various tradeoffs in terms of how much they optimise cost versus quality. This example demonstrates a simple default BootstrapFewShot Optimiser.

In [1]:
%env pip install -qU langchain-ibm

env: pip=install -qU langchain-ibm


In [2]:
import dspy
from dsp.utils import deduplicate
from dspy.datasets import HotPotQA
from dspy.predict.retry import Retry
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch
from dspy.evaluate.evaluate import Evaluate
from dspy.primitives.assertions import assert_transform_module, backtrack_handler

import os
import requests
from dsp import LM
import dspy

In [3]:
os.environ['WATSONX_URL']="https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
os.environ['WATSONX_APIKEY']=""
os.environ['WATSONX_PROJECTID']=""

### Create Implementation of DSpy LM Class for watsonx.ai models

In [4]:
class Watson(LM):
    def __init__(self,model,api_key):
        self.kwargs = {
            "model": model,
            "temperature": 0.0,
            "max_tokens": 150,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
            "n": 1,
        }
        self.model = model
        self.api_key = api_key
        self.provider = "default"
        self.history = []
        
        self.base_url = os.environ['WATSONX_URL']
        self.project_id = os.environ['WATSONX_PROJECTID']

    def basic_request(self, prompt: str, **kwargs):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Accept": "application/json",
            "content-type": "application/json"
        }

        data = {
            "parameters": {**kwargs},
            "model_id": self.model,
            "input": prompt,
            "project_id": self.project_id
        }

        response = requests.post(self.base_url, headers=headers, json=data)
        response = response.json()

        self.history.append({
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
        })
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.request(prompt, **kwargs)

        print(response)

        completions = [result["generated_text"] for result in response["results"]]

        return completions

### Configure DSpy with the watsonx.ai Language Model

In [5]:
import requests

def generate_access_token(api_key):
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": api_key
    }
    response = requests.post('https://iam.cloud.ibm.com/identity/token', data=data, headers=headers)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

token = generate_access_token(os.environ['WATSONX_APIKEY'])

In [6]:
watsonx = Watson(model="meta-llama/llama-2-70b-chat",api_key=token)
dspy.settings.configure(lm=watsonx, trace=[], temperature=0.7)
#dspy.settings.configure(lm=watsonx_llm, trace=[], temperature=0.7)

### Load the dataset

We make use of the mentioned HotPotQA dataset, a collection of complex question-answer pairs typically answered in a multi-hop fashion. We can load this dataset provided by DSPy through the HotPotQA class:

In [7]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(rm=colbertv2_wiki17_abstracts)

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

### Build the Signature of the DSpy Modules

Now that we have the data loaded, let's start defining the signatures for the sub-tasks of our pipeline.

We can identify our simple input question and output answer, but since we are building out a RAG pipeline, we wish to utilize some contextual information from our ColBERT corpus. So let's define our signature: context, question --> answer.

In [8]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

### Building the Pipeline

We will build our RAG pipeline as a DSPy module which will require two methods:

The __init__ method will simply declare the sub-modules it needs: dspy.Retrieve and dspy.ChainOfThought. The latter is defined to implement our GenerateAnswer signature.
The forward method will describe the control flow of answering the question using the modules we have: Given a question, we'll search for the top-3 relevant passages and then feed them as context for answer generation.

In [9]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

### Optimising the Pipeline

Compiling the RAG program
Having defined this program, let's now compile it. Compiling a program will update the parameters stored in each module. In our setting, this is primarily in the form of collecting and selecting good demonstrations for inclusion within the prompt(s).

Compiling depends on three things:

A training set. We'll just use our 20 question–answer examples from trainset above.
A metric for validation. We'll define a simple validate_context_and_answer that checks that the predicted answer is correct and that the retrieved context actually contains the answer.
A specific teleprompter. The DSPy compiler includes a number of teleprompters that can optimize your programs.

In [10]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

  0%|          | 0/20 [00:00<?, ?it/s]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:06.163Z', 'results': [{'generated_text': ' find the answer. We know that At My Window is an album released in 1987', 'generated_token_count': 20, 'input_token_count': 956, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}


  5%|▌         | 1/20 [00:03<01:07,  3.55s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:07.396Z', 'results': [{'generated_text': ' Townes Van Zandt', 'generated_token_count': 7, 'input_token_count': 980, 'stop_reason': 'eos_token'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:10.099Z', 'results': [{'generated_text': ' find the answer. We know that Candace Kita guest starred in "The Wayans B', 'generated_token_count': 20, 'input_token_count': 1319, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose

 10%|█         | 2/20 [00:07<01:12,  4.03s/it]

...

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:46.599Z', 'results': [{'generated_text': ' Buena Vista Distribution Company', 'generated_token_count': 7, 'input_token_count': 1229, 'stop_reason': 'eos_token'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:49.387Z', 'results': [{'generated_text': ' find the answer. We know that Samantha Cristoforetti is the first Italian woman in', 'generated_token_count': 20, 'input_token_count': 1030, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-par

 65%|██████▌   | 13/20 [00:47<00:26,  3.74s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:51.114Z', 'results': [{'generated_text': ' Space\n\n---\n\nContext:\n[1] «The Great Fire of London | The', 'generated_token_count': 20, 'input_token_count': 1054, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:53.258Z', 'results': [{'generated_text': ' answer this question. We know that Eric Davis played for the Los Angeles Dodgers, and we', 'generated_token_count': 20, 'input_token_count': 1411, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a No

 70%|███████   | 14/20 [00:50<00:21,  3.61s/it]

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:54.462Z', 'results': [{'generated_text': ' Outfield of Dreams', 'generated_token_count': 6, 'input_token_count': 1435, 'stop_reason': 'eos_token'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
Bootstrapped 4 full traces after 15 examples in round 0.


Teleprompters: Teleprompters are powerful optimizers that can take any program and learn to bootstrap and select effective prompts for its modules. Hence the name which means "prompting at a distance".

Different teleprompters offer various tradeoffs in terms of how much they optimize cost versus quality, etc. We will used a simple default BootstrapFewShot in the example above.

If you're into analogies, you could think of this as your training data, your loss function, and your optimizer in a standard DNN supervised learning setup. Whereas SGD is a basic optimizer, there are more sophisticated (and more expensive!) ones like Adam or RMSProp.

### Executing the Pipeline
Now that we've compiled our RAG program, let's try it out.

In [11]:
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:57.191Z', 'results': [{'generated_text': ' answer this question. We know that David Gregory was a Scottish physician and inventor, and we', 'generated_token_count': 20, 'input_token_count': 3436, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:10:59.431Z', 'results': [{'generated_text': ' Kinnairdy Castle\n\n---\n\nContext:\n[1] «Johnny De', 'generated_token_count': 20, 'input_token_count': 3460, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IB

In [12]:
watsonx.history

[{'prompt': 'Answer questions with short factoid answers.\n\n---\n\nQuestion: Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where?\nAnswer: space\n\nQuestion: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?\nAnswer: "Outfield of Dreams"\n\nQuestion: which American actor was Candace Kita guest starred with\nAnswer: Bill Murray\n\nQuestion: Tombstone stared an actor born May 17, 1955 known as who?\nAnswer: Bill Paxton\n\nQuestion: The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas founded in what year?\nAnswer: 2010\n\nQuestion: Which is taller, the Empire State Building or the Bank of America Tower?\nAnswer: The Empire State Building\n\nQuestion: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?\nAnswer: The Wa

Even though we haven't written any of this detailed demonstrations, we see that DSPy was able to bootstrap this 3,000 token prompt for 3-shot retrieval-augmented generation with hard negative passages and uses Chain-of-Thought reasoning within an extremely simply-written program.

This illustrates the power of composition and learning. Of course, this was just generated by a particular teleprompter, which may or may not be perfect in each setting. As you'll see in DSPy, there is a large but systematic space of options you have to optimize and validate with respect to your program's quality and cost.

You can also easily inspect the learned objects themselves.

In [13]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

generate_answer
Example({'augmented': True, 'context': ['Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuyver.', "Kwon Tae-man | Kwon Tae-man (born 1941) was an early Korean hapkido practitioner and a pioneer of the art, first in Korea and then in the United States. He formed one of the earliest dojang's for hapkido in the United States in Torrance, California, and has been featured in many magazine articles promoting the art.", 'Hee Il Cho | Cho Hee Il (born October 13, 1940) is a prominent Korean-American master of taekwondo, holding the rank of 9th "dan" in the martial art. He has written 11 martial art books, produced 70 martial art tra

### Evaluating the Pipeline

We can now evaluate our compiled_rag program on the dev set. Of course, this tiny set is not meant to be a reliable benchmark, but it'll be instructive to use it for illustration.

Let's evaluate the accuracy (exact match) of the predicted answer.

In [14]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=False, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:11:01.350Z', 'results': [{'generated_text': ' answer this question. We know that Cangzhou is a prefecture-level city in', 'generated_token_count': 20, 'input_token_count': 3326, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'id': 'disclaimer_warning', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}
{'model_id': 'meta-llama/llama-2-70b-chat', 'created_at': '2024-04-08T10:11:03.777Z', 'results': [{'generated_text': ' Yes\n\n---\n\nContext:\n[1] «The 1989 film', 'generated_token_count': 20, 'input_token_count': 3350, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-

### Evaluating the Retrieval

It may also be instructive to look at the accuracy of retrieval. While there are multiple ways to do this, we can simply check whether the retrieved passages contain the answer.

We can make use of our dev set which includes the gold titles that should be retrieved.

In [ ]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)

compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

Although this simple compiled_rag program is able to answer a decent fraction of the questions correctly (on this tiny set, over 40%), the quality of retrieval is much lower.

This potentially suggests that the LM is often relying on the knowledge it memorized during training to answer questions. To address this weak retrieval, let's explore a second program that involves more advanced search behavior.